In [1]:
import torch
import imgaug.augmenters as iaa
import numpy as np

from architectures import UNet
from dataset import dataset_gen
import gpu_manager as gm
import oper
from stats import print_data

## 1. Paths

In [2]:
train_path = '../RibFrac/preprocessed/train/'
val_path = '../RibFrac/preprocessed/val/'

model_path = 'saved_models/baseline.tar'
save_path = 'saved_models/baseline_16.tar'
stats_path = 'stats/baseline_16'

## 2. Fix Backend (for reproducing results)

In [3]:
seed = 12
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True 
torch.backends.cudnn.benchmark = False

## 3. GPU Test

In [4]:
if torch.cuda.is_available():
    print('GPU is available.')
    manager = gm.GPUManager()
    device = torch.device(manager.auto_choice())
else:
    print('GPU is not available. Use CPU instead.')
    device = torch.device('cpu')

GPU is available.
Choosing the GPU device has largest free memory...
Sorted by free memory size
Using GPU 0:
index:0
gpu_name:GeForce RTX 2080 Ti
memory.free:10979
memory.total:10989
power.draw:19
power.limit:250
specified:True


## 4. Data Preprocessing and Augmentation

In [5]:
seqs_train = []
seqs_train.append(iaa.Sequential([
    iaa.size.Resize(128, interpolation='nearest'),
]))
seqs_train.append(iaa.Sequential([
    iaa.flip.Fliplr(1.0),
    iaa.size.Resize(128, interpolation='nearest'),
]))
seqs_train.append(iaa.Sequential([
    iaa.geometric.Affine(rotate=(-15,15), order=0),
    iaa.size.Resize(128, interpolation='nearest'),
]))
seqs_train.append(iaa.Sequential([
    iaa.geometric.Affine(scale=(0.8,1.2), order=0),
    iaa.size.Resize(128, interpolation='nearest'),
]))
seqs_train.append(iaa.Sequential([
    iaa.geometric.Affine(translate_px=(-32,32)),
    iaa.size.Resize(128, interpolation='nearest'),
]))

seqs_val = [iaa.Sequential([
    iaa.size.Resize(128, interpolation='nearest'),
])]

train_dataset = dataset_gen(train_path, seqs_train)
val_dataset = dataset_gen(val_path, seqs_val)

print('train size:', len(train_dataset))
print('val size:', len(val_dataset))

train size: 770635
val size: 26420


## 5. Reload
### 5.1 Model Params (keep consistent with the best model)

In [6]:
in_channels=1
n_classes=2
depth=5
wf=6 
padding=True
batch_norm=False
up_mode='upconv'

"""
Args:
    in_channels (int): number of input channels
    n_classes (int): number of output channels
    depth (int): depth of the network
    wf (int): number of filters in the first layer is 2**wf
    padding (bool): if True, apply padding such that the input shape
                    is the same as the output.
                    This may introduce artifacts
    batch_norm (bool): Use BatchNorm after layers with an
                       activation function
    up_mode (str): one of 'upconv' or 'upsample'.
                   'upconv' will use transposed convolutions for
                   learned upsampling.
                   'upsample' will use bilinear upsampling.
"""


"\nArgs:\n    in_channels (int): number of input channels\n    n_classes (int): number of output channels\n    depth (int): depth of the network\n    wf (int): number of filters in the first layer is 2**wf\n    padding (bool): if True, apply padding such that the input shape\n                    is the same as the output.\n                    This may introduce artifacts\n    batch_norm (bool): Use BatchNorm after layers with an\n                       activation function\n    up_mode (str): one of 'upconv' or 'upsample'.\n                   'upconv' will use transposed convolutions for\n                   learned upsampling.\n                   'upsample' will use bilinear upsampling.\n"

### 5.2 Checkpoint Reload

In [7]:
checkpoint = torch.load(model_path)

model = UNet(in_channels=in_channels, 
            n_classes=n_classes, 
            depth=depth, 
            wf=wf, 
            padding=padding, 
            batch_norm=batch_norm, 
            up_mode=up_mode).to(device)
model.load_state_dict(checkpoint['model_state_dict'])

optim = torch.optim.Adam(model.parameters())
optim.load_state_dict(checkpoint['optim_state_dict'])

best_epoch = checkpoint['epoch']
loss = checkpoint['loss']

print('best_epoch:', best_epoch)
print('min_val_loss:', loss)

best_epoch: 3
min_val_loss: 0.000623614717467576


### 5.3 Loss function

In [8]:
loss_func = 'focal_loss'
gamma = 0
alpha = 0.75

## 6. Model Training and Evaluation

In [ ]:
epochs = 16 - best_epoch
pad = 0
batch_size = 64

stats = oper.run_model(
            device = device,
            train_dataset = train_dataset, 
            val_dataset = val_dataset,
            model = model,
            optim = optim, 
            loss_func=loss_func,
            gamma = gamma, 
            alpha = alpha,
            epochs = epochs,
            pad = pad,
            batch_size = batch_size,
            save_path = save_path)

start running at:  Mon Jul 13 22:22:33 2020
epoch: 1
start at:  Mon Jul 13 22:22:33 2020


## 7. Statistics

In [ ]:
print_data(
    epochs = epochs, 
    stats = stats, 
    stats_path = stats_path)